In [2]:
# import all the libraries we need
from datetime import datetime

import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense


In [4]:
# For visualizing a row of the dataset, transforming it into 3x3 image form
def visualize_img(img_vec, title=""):
    plt.imshow(img_vec.values.reshape((3,3)), cmap="hot")
    plt.title(title)
    plt.show()
# To add a timestamp to a string of text
def add_timestamp(text):
    return text + str(datetime.now().year) \
           + str(datetime.now().month) \
           + str(datetime.now().day) \
           + str(datetime.now().second)
def load_data(filename):
    xs = []
    ys = []
    data = pd.read_pickle(filename)
    for x, y in data:
        t = np.array([d for d in x])
        inx = int()
        for i in range(len(x)):
            if x[i] != y[i]:
                inx = i
                break
        xs.append(t)
        ys.append(inx)
    return xs, ys

In [6]:
# load data from text files
x, y = load_data('TicTacNet/data/randombased_all75704.pkl')
test, test_label = load_data('TicTacNet/data/all22936.pkl')

In [8]:
# convert to dataframes
x = pd.DataFrame(x)
y = pd.DataFrame(y)
test = pd.DataFrame(test)
test_label = pd.DataFrame(test_label)

In [10]:
# Convert the y data to [0 0 1 0 ... 0] form
y = keras.utils.to_categorical(y, num_classes=9)
test_label = keras.utils.to_categorical(test_label, num_classes=9)

In [11]:
# get a feel for what data in in the training set
print(x.describe())

                  0             1             2             3             4  \
count  75704.000000  75704.000000  75704.000000  75704.000000  75704.000000   
mean       0.024939     -0.040275      0.022694     -0.036299      0.054132   
std        0.583388      0.555069      0.585266      0.556282      0.591699   
min       -1.000000     -1.000000     -1.000000     -1.000000     -1.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

                  5             6             7            8  
count  75704.000000  75704.000000  75704.000000  75704.00000  
mean      -0.037528      0.024213     -0.034595      0.02272  
std        0.553379      0.585735      0.552519      0.58461  
min       -1.000000     -1.000000   

In [11]:
# to visualize dataset
random_indices = np.random.randint(0, x.shape[0], 3)  
for idx in random_indices:
    visualize_img(x.iloc[idx, :], title=str(idx))

In [14]:
# gather means and std's
x_means = x.mean(axis=0)
x_stds = x.std(axis=0)
test_means = test.mean(axis=0)
test_stds = test.std(axis=0)

In [16]:
# Make training and testing set have Zero mean
# and 1 standard deviation
x = x.subtract(x_means)
x = x.divide(x_stds).fillna(0)
test = test.subtract(test_means)
test = test.divide(test_stds).fillna(0)

In [18]:
# build the model
model = Sequential()
model.add(Dense(9, activation='relu', input_dim=9))
model.add(Dense(9, activation='softmax'))
# I went with the simplest model i could. I wanted to try to keep max
# accuracy above 94% and have a fast training time
model.compile(loss='categorical_crossentropy',      # A way to compare outputs of categorical problems where each is in a range [0,1]
              optimizer='adam',                     # a good optimizer
              metrics=['accuracy'])                 # use the model accuracy for training
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 90        
Total params: 180
Trainable params: 180
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Train the model for a fixed amount of epochs
model.fit(x.as_matrix(), y,
          epochs=5)

C:\Users\Cameron\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


ValueError: Error when checking target: expected dense_4 to have 2 dimensions, but got array with shape (75704, 9, 9)

In [27]:
# Check the accuracy of the model on the test set
score = model.evaluate(test.as_matrix(), test_label)
print('Loss: {0} Accuracy {1}'.format(score[0], score[1]))

C:\Users\Cameron\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


ValueError: Error when checking target: expected dense_4 to have 2 dimensions, but got array with shape (22936, 9, 9)

In [23]:
# see what the network weights look like
for layer in model.layers:
    weights = layer.get_weights()  # list of numpy arrays
    print(weights)

[array([[ 0.46529818,  0.32321233, -0.4870154 , -0.39765137, -0.21493259,
        -0.22949055, -0.11731461, -0.10649991,  0.06715143],
       [ 0.40427554, -0.33177078, -0.04868966, -0.00725144, -0.33728907,
         0.22846615, -0.16969606, -0.3307354 , -0.52740246],
       [-0.36577705, -0.3697852 , -0.52908015,  0.47732508, -0.449371  ,
        -0.05832869, -0.20991603, -0.18119967,  0.2637878 ],
       [-0.0110783 ,  0.3008446 , -0.44959524, -0.25615054,  0.00401789,
        -0.43883604, -0.5141346 ,  0.11448175,  0.19849044],
       [ 0.0061065 , -0.3732574 , -0.05427152,  0.46052897,  0.41804767,
         0.56751096, -0.03602779, -0.42792842,  0.09468573],
       [ 0.26089287,  0.08746552, -0.35980546, -0.35543698, -0.21244481,
        -0.26598266,  0.22557867, -0.16994134, -0.07781497],
       [-0.12465525,  0.48220932, -0.16661555, -0.02636939,  0.14939761,
         0.21551073,  0.19302142, -0.0050379 , -0.05154425],
       [ 0.3622616 , -0.40732646, -0.43330383,  0.47284448, -

In [25]:
# save the model for later use
filename = 'TicTacNet/models/tictacModel'
model.save(add_timestamp(filename))